In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as f
import numpy as np
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
sc = spark.sparkContext

In [10]:
df = spark.read.csv("/data/titanic.csv", header=True)
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [30]:
null = df.filter(f.col("Cabin").isNull()).groupBy("gender").count()
null.show()

+------+-----+
|gender|count|
+------+-----+
|female|  217|
|  male|  470|
+------+-----+



In [25]:
avg_age = df.agg(f.avg("age"))
avg_age.show()

+-----------------+
|         avg(age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [29]:
null_fill = df.fillna({"Age" : 29.7})
null_fill.show(20)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|29.7|    0|    0|      

In [56]:

survived = df.filter(f.col("Survived") == 1).count()
didnt_survive = df.filter(f.col("Survived") == 0).count()
print(f"survived: {survived} \ndidnt survive: {didnt_survive}")



survived: 342 
didnt survive: 549


In [59]:

top5 = df.groupBy("Embarked").count().sort(f.col("count").desc()).show(5)
top5


+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



In [65]:
df.groupBy('Pclass').agg(f.avg('Survived').alias("SurvivalRate")).show()


+------+-------------------+
|Pclass|       SurvivalRate|
+------+-------------------+
|     3|0.24236252545824846|
|     1| 0.6296296296296297|
|     2|0.47282608695652173|
+------+-------------------+



In [71]:
df.agg(f.max("Fare")).show()
df.agg(f.min("Fare")).show()
df.agg(f.avg("Fare")).show()



+---------+
|max(Fare)|
+---------+
|     93.5|
+---------+

+---------+
|min(Fare)|
+---------+
|        0|
+---------+

+----------------+
|       avg(Fare)|
+----------------+
|32.2042079685746|
+----------------+



In [74]:

age_0_18 = df.filter(df['Age'] <= 18).count()
age_19_35 = df.filter(  (df['Age'] >=19)  & (df['Age'] <= 35)).count()
age_36_60 = df.filter(  (df['Age'] >=36)  & (df['Age'] <= 60)).count()
age_higher = df.filter(df['Age'] > 60).count()

print(f"0-18: {age_0_18}, 19-35: {age_19_35}, 36-60: {age_36_60}, 60+: {age_higher}")

0-18: 139, 19-35: 358, 36-60: 195, 60+: 22


In [96]:
df.write.option("header", True).mode("overwrite").csv("/data/processed_output/df")
null.write.option("header", True).mode("overwrite").csv("/data/processed_output/null")
null_fill.write.option("header", True).mode("overwrite").csv("/data/processed_output/null_fill")


Py4JJavaError: An error occurred while calling o729.csv.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=jovyan, access=WRITE, inode="/titanic_lab":root:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:399)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:255)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:193)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1879)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1863)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1822)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp.mkdirs(FSDirMkdirOp.java:59)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:3233)
	at org.apache.hadoop